<a href="https://colab.research.google.com/github/Munfred/scdefg/blob/main/scdefg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone repo which contains the html templates for flask
!git clone https://github.com/Munfred/scdefg.git

Cloning into 'scdefg'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [4]:
# install flask dependencies on colab, takes a minute or two
!pip install scvi-tools --quiet
!pip install Flask-Misaka --quiet
!pip install Flask-Markdown --quiet
!pip install flask-ngrok --quiet
!pip install flask==0.12.2 --quiet
# Newer versions of flask don't work in Colab
# See https://github.com/plotly/dash/issues/257

In [18]:
from flask import Flask, Response, jsonify, request, render_template, Blueprint, send_file, redirect, url_for
import logging
import pandas as pd
import sys
import json
import time
from io import StringIO
import urllib
from flaskext.markdown import Markdown
from flask_misaka import Misaka
from flask_ngrok import run_with_ngrok
from google.colab import files
import scvi
print('scvi-tools verseion:', scvi.__version__)

scvi-tools verseion: 0.8.1


In [34]:
logging.basicConfig(level=logging.INFO)
logger=logging.getLogger(__name__)
logger.info('Starting wormcells-de...')

flask_app = Flask(__name__, template_folder='./scdefg/templates', static_folder='./scdefg/static')
flask_app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

run_with_ngrok(flask_app)
Misaka(flask_app, math_explicit = True)


tables = Blueprint('tables', __name__, url_prefix='/tables')

# df with the number of cells of each label in each dataset
df = pd.read_csv(flask_app.open_resource('./scdefg/df.csv'))

# to render the table titles better we replace underscores with spaces,
# use non breaking hyphens (&#8209;) and say batch1 instead of just 1
df_nice_names = df.copy()
df_nice_names.columns = df_nice_names.columns.str.replace('_',' ')
df_nice_names.columns = df_nice_names.columns.str.replace('cho-1 1','cho-1 batch1')
df_nice_names.columns = df_nice_names.columns.str.replace('cho-1 2','cho-1 batch2')
df_nice_names.columns = df_nice_names.columns.str.replace('unc-47 2','unc-47 batch2')
df_nice_names.columns = df_nice_names.columns.str.replace('unc-47 1','unc-47 batch1')
df_nice_names.columns = df_nice_names.columns.str.replace('-','&#8209;')

# same for cell type names
# df_nice_names['Cell Type']= df_nice_names['Cell Type'].str.replace('_',' ')


# convert df to dict for sending as json to datatables
dict_df = df_nice_names.to_dict(orient='records')
# convert column names into dict for sending as json to datatables
columns = [{"data": item, "title": item} for item in df_nice_names.columns]

#### datatables ####

@tables.route("/", methods=['GET'])
def clientside_table_content():
    return jsonify({'data': dict_df, 'columns': columns})

flask_app.register_blueprint(tables)

@flask_app.route("/")
def clientside_table():
    return render_template("clientside_table.html")


####

@flask_app.route("/test")
def test():
    return render_template("test.html")

# @flask_app.route("/")
# def index():
#     logger.info('Got a request for index!')
#     return render_template("index.html")

@flask_app.route('/submit', methods=['POST', 'GET'])
def receive_submission():
    logger.info('Got a submission!')
    print('PRESSED THE SUBMISSION BUTTOOOON')
    # answer is a dict of json strings containing selected row and column index numbers
    answer = request.form.to_dict(flat=False)
    print(answer)
    print(df.head())

    
    # need to convert the json strings to dict, then to a data frame
    # data1 is the selection for the first group, data2 for the second
    data1 = json.loads(answer['data1'][0])
    data1_df = pd.DataFrame.from_dict(data1[0])

    print(data1_df)
    data2 = json.loads(answer['data2'][0])
    data2_df = pd.DataFrame.from_dict(data2[0])

    # now map the index number to experiment name and cell type name
    group1_df = pd.DataFrame()
    group1_df['cell_type1'] = data1_df['row'].map(df['Cell Type'])
    group1_df['experiment1'] = data1_df['column'].map(pd.Series(df.columns.values))

    group2_df = pd.DataFrame()
    group2_df['cell_type2'] = data2_df['row'].map(df['Cell Type'])
    group2_df['experiment2'] = data2_df['column'].map(pd.Series(df.columns.values))

    genes = StringIO(json.loads(answer['genes'][0]))
    genes_df = pd.read_csv(genes, names=['selected_genes'])

    jobname_df = pd.DataFrame(columns=['job_name'])
    jobname_df = jobname_df.append({'job_name': 'bla'}, ignore_index=True)

    selected_groups_df = pd.concat([group1_df, group2_df, genes_df, jobname_df], axis=1)
    print(selected_groups_df)

    csv_buffer = StringIO()
    selected_groups_df.to_csv(csv_buffer)
    # files.download("readme.md")


    # return 'derpderp'
    print('RETURN FILE NOW')
    # return send_file('/content/df.csv')
    # return send_file('./foo.txt', as_attachment=True)
    return redirect(url_for('test'))

@flask_app.route("/get_results_csv")
def get_results_csv():
    # csv = pd.read_csv('results.csv')
    with open("results.csv") as fp:
        csv = fp.read()  
    # csv = '1,2,3\n4,5,6\n'
    return Response(
        csv,
        mimetype="text/csv",
        headers={"Content-disposition":
                 "attachment; filename=results.csv"})

@flask_app.route('/file_downloads')
def file_downloads():
	try:
		return render_template("""
			<body class="body">
				<div class="container" align="left">
					<a href="/return_files/" target="blank"><button class='btn btn-default'>Download!</button></a>
				</div>
			</body>	
		""")
	except Exception as e:
		return str(e)
  
@flask_app.route('/return_files/')
def return_files():
	try:
		return send_file('./foo.txt', as_attachment=True)#, attachment_filename='python.jpg')
	except Exception as e:
		return str(e)



if __name__ == "__main__":
    flask_app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2857a7cbec38.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:38:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:38:44] "GET /static/js/clientside_table.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:38:44] "GET /tables/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:38:45] "GET /static/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:38:45] "GET /tables/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:41:24] "POST /submit HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:41:24] "GET /test HTTP/1.1" 200 -


PRESSED THE SUBMISSION BUTTOOOON
{'data1': ['[[{"row":9,"column":3}]]'], 'data2': ['[[{"row":0,"column":3}]]'], 'genes': ['"nlp\\r\\nflp\\r\\nins"']}
             Cell Type  L2_experiment_1  ...  tph-1_ceh-10  unc-3
0     ABarpaaa_lineage                0  ...             0      0
1                 ADA?                0  ...             1     62
2  ADA_ADE_grandparent                0  ...             0      0
3                  ADE                0  ...             1      2
4          ADE_CEP_PDE              126  ...             0      0

[5 rows x 22 columns]
   row  column
0    9       3
   cell_type1            experiment1  ... selected_genes job_name
0  ADL_parent  Waterston_300_minutes  ...            nlp      bla
1         NaN                    NaN  ...            flp      NaN
2         NaN                    NaN  ...            ins      NaN

[3 rows x 6 columns]
RETURN FILE NOW


INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:41:24] "GET /test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:41:25] "GET /static/js/submission_logic.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2021 02:41:28] "GET /get_results_csv HTTP/1.1" 200 -


In [23]:
!ls

results.csv  sample_data  scdefg


In [27]:
pd.read_csv('results.csv')

,Unnamed: 0,Cell Type,L2_experiment_1,L2_experiment_2,Waterston_300_minutes,Waterston_400_minutes,Waterston_500_minutes_batch_1,Waterston_500_minutes_batch_2,Murray_r17,Murray_b01,Murray_b02,acr-2,ceh-34,cho-1_1,cho-1_2,ift-20,eat-4,unc-47_2,unc-47_1,nmr-1,Pan,tph-1_ceh-10,unc-3
0,0,ABarpaaa_lineage,0,0,92,96,2,0,41,28,14,0,0,0,0,0,0,0,0,0,0,0,0
1,1,ADA?,0,0,0,0,0,0,0,0,0,7,1,2,2,1,124,4,0,3,168,1,62
2,2,ADA_ADE_grandparent,0,0,6,32,0,0,2,7,3,0,0,0,0,0,0,0,0,0,0,0,0
3,3,ADE,0,0,1,3,46,16,9,9,6,1,1,0,1,50,1,0,0,0,109,1,2
4,4,ADE_CEP_PDE,126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,337,Vulval_precursor_bin_1_late_L2,282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
338,338,Vulval_precursor_bin_2_early_L3,271,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
339,339,XXX,73,0,0,1,6,3,1,8,6,0,0,0,0,0,0,0,0,0,0,0,0
340,340,Y,0,0,2,18,6,10,1,4,2,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
df.to_csv('results.csv')

In [35]:

!zip -r scdefg.zip scdefg/


  adding: scdefg/ (stored 0%)
  adding: scdefg/LICENSE (deflated 43%)
  adding: scdefg/templates/ (stored 0%)
  adding: scdefg/templates/clientside_table.html (deflated 68%)
  adding: scdefg/templates/test.html (deflated 50%)
  adding: scdefg/templates/.ipynb_checkpoints/ (stored 0%)
  adding: scdefg/templates/introduction.txt (deflated 42%)
  adding: scdefg/templates/template.html (deflated 73%)
  adding: scdefg/.git/ (stored 0%)
  adding: scdefg/.git/info/ (stored 0%)
  adding: scdefg/.git/info/exclude (deflated 28%)
  adding: scdefg/.git/refs/ (stored 0%)
  adding: scdefg/.git/refs/tags/ (stored 0%)
  adding: scdefg/.git/refs/heads/ (stored 0%)
  adding: scdefg/.git/refs/heads/main (stored 0%)
  adding: scdefg/.git/refs/remotes/ (stored 0%)
  adding: scdefg/.git/refs/remotes/origin/ (stored 0%)
  adding: scdefg/.git/refs/remotes/origin/HEAD (stored 0%)
  adding: scdefg/.git/objects/ (stored 0%)
  adding: scdefg/.git/objects/info/ (stored 0%)
  adding: scdefg/.git/objects/c6/ (stored